**Importation of libraries**

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import scipy.optimize as optimize
import requests
import io

In [2]:
#Constantes
date_format = '%d/%m/%Y'

**Importation of Yield Curve Euro + Cleaning**

In [74]:
DATAFLOW = "YC"
FREQ = "B"  # Daily - businessweek
REF_AREA = "U2"  # Euro area (changing composition)
CURRENCY = "EUR"
PROVIDER_FM = "4F"  # ECB
INSTRUMENT_FM = "G_N_A"  # Government bond, nominal, all issuers whose rating is triple A
PROVIDER_FM_ID = "SV_C_YM"  # Svensson model - continuous compounding - yield error minimisation
DATA_TYPE_FM = "SR_1M+SR_3M+SR_1Y+SR_5Y+SR_10Y"

#N_OBS = 600
#PARAMETERS = f"lastNObservations={N_OBS}&detail=dat"
url = f"https://data-api.ecb.europa.eu/service/data/{DATAFLOW}/{FREQ}.{REF_AREA}.{CURRENCY}.{PROVIDER_FM}.{INSTRUMENT_FM}.{PROVIDER_FM_ID}.{DATA_TYPE_FM}"

response = requests.get(url, headers={'Accept': 'text/csv'})
response.raise_for_status()
df = pd.read_csv(io.StringIO(response.text), parse_dates=['TIME_PERIOD'], index_col=['TIME_PERIOD'])

In [75]:
df = df[["KEY", "OBS_VALUE"]].pivot(columns="KEY", values="OBS_VALUE").rename(columns = {"YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y" : "10 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_5Y" : "5 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_1Y" : "1 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_3M" : "3 Mo"}).reindex(columns=["3 Mo", "1 Yr", "5 Yr", "10 Yr"])

df.index = df.index.strftime(date_format)
df.index.names = ['Date']
df.reset_index

<bound method DataFrame.reset_index of KEY             3 Mo      1 Yr      5 Yr     10 Yr
Date                                              
06/09/2004  2.034172  2.298838  3.457222  4.209220
07/09/2004  2.040893  2.328891  3.479518  4.209626
08/09/2004  2.044384  2.346666  3.507894  4.228419
09/09/2004  2.037111  2.308988  3.430628  4.161872
10/09/2004  2.034645  2.271566  3.374729  4.120981
...              ...       ...       ...       ...
10/11/2023  3.756260  3.494435  2.623349  2.755196
13/11/2023  3.721776  3.487610  2.658202  2.788169
14/11/2023  3.701341  3.446597  2.581791  2.684807
15/11/2023  3.756451  3.420655  2.566103  2.687105
16/11/2023  3.793927  3.392231  2.524870  2.634927

[4911 rows x 4 columns]>

**Importation of yield curve Dollar + cleaning**

In [76]:
dftaux_Dollar = pd.read_csv(filepath_or_buffer = './Courbe de taux 2023 $.csv',sep = ',', parse_dates=['Date'], index_col=['Date']).sort_index(ascending=True)

In [77]:
dftaux_Dollar.index = dftaux_Dollar.index.strftime(date_format)
dftaux_Dollar.head(10)

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
03/01/2023,4.17,4.42,4.53,4.70,4.77,4.72,4.40,4.18,3.94,3.89,3.79,4.06,3.88
04/01/2023,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81
05/01/2023,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
06/01/2023,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
09/01/2023,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
10/01/2023,4.41,4.62,4.73,4.77,4.85,4.74,4.24,3.94,3.72,3.67,3.61,3.91,3.74
11/01/2023,4.42,4.62,4.72,4.82,4.84,4.73,4.20,3.90,3.66,3.61,3.54,3.84,3.67
12/01/2023,4.57,4.59,4.66,4.74,4.76,4.66,4.12,3.79,3.53,3.48,3.43,3.73,3.56
13/01/2023,4.58,4.59,4.67,4.73,4.77,4.69,4.22,3.88,3.60,3.55,3.49,3.79,3.61


In [78]:
dftaux_Dollar = dftaux_Dollar[['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']]
dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr
Date,,,,,
03/01/2023,4.17,4.53,4.72,3.94,3.79
04/01/2023,4.20,4.55,4.71,3.85,3.69
05/01/2023,4.30,4.66,4.78,3.90,3.71
06/01/2023,4.32,4.67,4.71,3.69,3.55
09/01/2023,4.37,4.70,4.69,3.66,3.53
10/01/2023,4.41,4.73,4.74,3.72,3.61
11/01/2023,4.42,4.72,4.73,3.66,3.54
12/01/2023,4.57,4.66,4.66,3.53,3.43
13/01/2023,4.58,4.67,4.69,3.60,3.49


In [79]:
for x in ['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']:
    dftaux_Dollar[f'Return_YC_Euro_{x}'] = dftaux_Dollar[x] - dftaux_Dollar[x].shift(1)
dftaux_Dollar.dropna(inplace=True)

dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Euro_1 Mo,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr
Date,,,,,,,,,,
04/01/2023,4.20,4.55,4.71,3.85,3.69,0.03,0.02,-0.01,-0.09,-0.10
05/01/2023,4.30,4.66,4.78,3.90,3.71,0.10,0.11,0.07,0.05,0.02
06/01/2023,4.32,4.67,4.71,3.69,3.55,0.02,0.01,-0.07,-0.21,-0.16
09/01/2023,4.37,4.70,4.69,3.66,3.53,0.05,0.03,-0.02,-0.03,-0.02
10/01/2023,4.41,4.73,4.74,3.72,3.61,0.04,0.03,0.05,0.06,0.08
11/01/2023,4.42,4.72,4.73,3.66,3.54,0.01,-0.01,-0.01,-0.06,-0.07
12/01/2023,4.57,4.66,4.66,3.53,3.43,0.15,-0.06,-0.07,-0.13,-0.11
13/01/2023,4.58,4.67,4.69,3.60,3.49,0.01,0.01,0.03,0.07,0.06
17/01/2023,4.60,4.71,4.67,3.60,3.53,0.02,0.04,-0.02,0.00,0.04


**Importation of stock indices from Yahoo Finance + Cleaning**

S&P 500


In [80]:
sp500_data = yf.download('^GSPC', start='2013-10-24', end='2023-10-24', progress=False)
sp500_data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-10-24,1747.479980,1753.939941,1745.500000,1752.069946,1752.069946,3671700000
2013-10-25,1756.010010,1759.819946,1752.449951,1759.770020,1759.770020,3175720000
2013-10-28,1759.420044,1764.989990,1757.670044,1762.109985,1762.109985,3282300000
2013-10-29,1762.930054,1772.089966,1762.930054,1771.949951,1771.949951,3358460000
2013-10-30,1772.270020,1775.219971,1757.239990,1763.310059,1763.310059,3523040000
2013-10-31,1763.239990,1768.530029,1755.719971,1756.540039,1756.540039,3826530000
2013-11-01,1758.699951,1765.670044,1752.699951,1761.640015,1761.640015,3686290000
2013-11-04,1763.400024,1768.780029,1761.560059,1767.930054,1767.930054,3194870000
2013-11-05,1765.670044,1767.030029,1755.760010,1762.969971,1762.969971,3516680000


In [81]:
print(sp500_data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [82]:
sp500_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
sp500_data = sp500_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'sp500'})

sp500_data['Date'] = pd.to_datetime(sp500_data['Date']).dt.strftime(date_format)

In [83]:
sp500_data.head(10)

,Date,sp500
0,24/10/2013,1752.069946
1,25/10/2013,1759.770020
2,28/10/2013,1762.109985
3,29/10/2013,1771.949951
4,30/10/2013,1763.310059
5,31/10/2013,1756.540039
6,01/11/2013,1761.640015
7,04/11/2013,1767.930054
8,05/11/2013,1762.969971
9,06/11/2013,1770.489990


Eurostoxx50

In [84]:
Eurostoxx50_data = yf.download('^STOXX50E', start='2013-10-24', end='2023-10-24', progress=False)
print(Eurostoxx50_data.head())
Eurostoxx50_data.size

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2013-10-24  3024.689941  3038.959961  3022.280029  3038.959961  3038.959961   
2013-10-25  3027.840088  3043.840088  3021.010010  3034.500000  3034.500000   
2013-10-28  3041.159912  3045.139893  3009.800049  3022.040039  3022.040039   
2013-10-29  3019.649902  3050.739990  3017.729980  3050.639893  3050.639893   
2013-10-30  3049.090088  3073.879883  3035.020020  3040.689941  3040.689941   

              Volume  
Date                  
2013-10-24  53819900  
2013-10-25  49404800  
2013-10-28  51393200  
2013-10-29  59850600  
2013-10-30  60923400  


15054

In [85]:
Eurostoxx50_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
Eurostoxx50_data = Eurostoxx50_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'Eurostoxx50'})

Eurostoxx50_data['Date'] = pd.to_datetime(Eurostoxx50_data['Date']).dt.strftime(date_format)

In [86]:
Eurostoxx50_data.head(10)

,Date,Eurostoxx50
0,24/10/2013,3038.959961
1,25/10/2013,3034.500000
2,28/10/2013,3022.040039
3,29/10/2013,3050.639893
4,30/10/2013,3040.689941
5,31/10/2013,3067.949951
6,01/11/2013,3052.139893
7,04/11/2013,3061.179932
8,05/11/2013,3035.919922
9,06/11/2013,3056.399902


**Euro/Dollar exchange rate importation + cleaning**

In [87]:
dftaux = pd.read_csv(filepath_or_buffer = './Taux de change euro dollar.csv',sep = ',')[['_TIME_PERIOD', '_OBS_VALUE']].rename(columns={'_TIME_PERIOD': 'Date', '_OBS_VALUE': 'TauxdeChange'})
dftaux['Date'] = pd.to_datetime(dftaux['Date']).dt.strftime(date_format)
dftaux.head(10)

,Date,TauxdeChange
0,04/01/1999,1.1789
1,05/01/1999,1.1790
2,06/01/1999,1.1743
3,07/01/1999,1.1632
4,08/01/1999,1.1659
5,11/01/1999,1.1569
6,12/01/1999,1.1520
7,13/01/1999,1.1744
8,14/01/1999,1.1653
9,15/01/1999,1.1626


In [88]:
dftaux = pd.merge(dftaux, sp500_data, on='Date', how='inner')
dftaux = pd.merge(dftaux, Eurostoxx50_data, on='Date', how='inner')
dftaux.set_index('Date', inplace=True)

In [89]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50
Date,,,
24/10/2013,1.3805,1752.069946,3038.959961
25/10/2013,1.3777,1759.770020,3034.500000
28/10/2013,1.3784,1762.109985,3022.040039
29/10/2013,1.3768,1771.949951,3050.639893
30/10/2013,1.3755,1763.310059,3040.689941
31/10/2013,1.3641,1756.540039,3067.949951
01/11/2013,1.3505,1761.640015,3052.139893
04/11/2013,1.3506,1767.930054,3061.179932
05/11/2013,1.3494,1762.969971,3035.919922


**Yields calculation**

In [90]:
dftaux['Log_Return_SP500'] = np.log(dftaux['sp500'] / dftaux['sp500'].shift(1))

In [91]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500
Date,,,,
24/10/2013,1.3805,1752.069946,3038.959961,NaN
25/10/2013,1.3777,1759.770020,3034.500000,0.004385
28/10/2013,1.3784,1762.109985,3022.040039,0.001329
29/10/2013,1.3768,1771.949951,3050.639893,0.005569
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847
01/11/2013,1.3505,1761.640015,3052.139893,0.002899
04/11/2013,1.3506,1767.930054,3061.179932,0.003564
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810


In [92]:
dftaux['Log_Return_Eurostoxx50'] = np.log(dftaux['Eurostoxx50'] / dftaux['Eurostoxx50'].shift(1))

In [93]:
dftaux = dftaux.dropna()

In [94]:

dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500,Log_Return_Eurostoxx50
Date,,,,,
25/10/2013,1.3777,1759.770020,3034.500000,0.004385,-0.001469
28/10/2013,1.3784,1762.109985,3022.040039,0.001329,-0.004115
29/10/2013,1.3768,1771.949951,3050.639893,0.005569,0.009419
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888,-0.003267
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847,0.008925
01/11/2013,1.3505,1761.640015,3052.139893,0.002899,-0.005167
04/11/2013,1.3506,1767.930054,3061.179932,0.003564,0.002957
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810,-0.008286
06/11/2013,1.3517,1770.489990,3056.399902,0.004256,0.006723


In [95]:
dftaux['Log_Return_TauxdeChange'] = np.log(dftaux['TauxdeChange'] / dftaux['TauxdeChange'].shift(1))

In [96]:
dftaux = dftaux.dropna()

In [97]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500,Log_Return_Eurostoxx50,Log_Return_TauxdeChange
Date,,,,,,
28/10/2013,1.3784,1762.109985,3022.040039,0.001329,-0.004115,0.000508
29/10/2013,1.3768,1771.949951,3050.639893,0.005569,0.009419,-0.001161
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888,-0.003267,-0.000945
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847,0.008925,-0.008322
01/11/2013,1.3505,1761.640015,3052.139893,0.002899,-0.005167,-0.010020
04/11/2013,1.3506,1767.930054,3061.179932,0.003564,0.002957,0.000074
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810,-0.008286,-0.000889
06/11/2013,1.3517,1770.489990,3056.399902,0.004256,0.006723,0.001703
07/11/2013,1.3365,1747.150024,3042.979980,-0.013270,-0.004400,-0.011309


***Generating scenarios with bootstrapping method***

Scenarios constants definition

In [113]:
nb_scenarios = 1000

Eurostoxx50 scenarios

In [ ]:
scenarios_bootstrap_Eurostoxx50 = []
for i in range(nb_scenarios):
    new_scenario = np.random.choice(dftaux['Log_Return_Eurostoxx50'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_Eurostoxx50.append(new_scenario)
print(scenarios_bootstrap_Eurostoxx50)

[array([ 0.00505826, -0.00116294, -0.00352572, ..., -0.00810452,
       -0.00292592,  0.00773148]), array([-0.00156947, -0.00713097, -0.01036038, ..., -0.00441633,
        0.00858789,  0.01324011]), array([-0.01522995, -0.00733637, -0.00845605, ...,  0.00608141,
       -0.00717028,  0.01605522]), array([-0.03819616, -0.00315255, -0.0025698 , ...,  0.01824176,
       -0.00766329,  0.00125025]), array([ 0.00650175,  0.00377636, -0.00103889, ..., -0.00127077,
        0.00516082,  0.00680371]), array([0.0224455 , 0.00502885, 0.00551469, ..., 0.00524948, 0.00118082,
       0.00596215]), array([ 0.01307703, -0.00600767, -0.01239082, ...,  0.01718951,
        0.01028543, -0.01090886]), array([ 0.02104156, -0.03679799, -0.00036371, ...,  0.00419994,
        0.00010232, -0.00414756]), array([ 0.00178735,  0.00692769, -0.00054349, ...,  0.00369009,
        0.01628297,  0.00224442]), array([ 0.00441028, -0.058874  , -0.00235678, ..., -0.00133411,
       -0.03091086, -0.00158177]), array([ 0.00865

In [ ]:
scenarios_totaux_Eurostoxx50 = sum(scenarios_bootstrap_Eurostoxx50)
print(scenarios_totaux_Eurostoxx50)

Exchange rates scenarios 

In [116]:
scenarios_bootstrap_TauxdeChange = []
for i in range(nb_scenarios):
    new_scenario = np.random.choice(dftaux['Log_Return_TauxdeChange'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_TauxdeChange.append(new_scenario)
print(scenarios_bootstrap_TauxdeChange)

[array([-0.0059824 , -0.00587519,  0.00090285, ..., -0.00036897,
       -0.00534736,  0.00271624]), array([ 5.56554399e-03,  4.57096163e-03,  8.26180797e-04, ...,
       -1.33264253e-03, -8.92578213e-05,  6.20870125e-04]), array([ 0.00901456,  0.00420514, -0.00374242, ...,  0.00768589,
        0.00608483,  0.00289413]), array([ 0.00558258,  0.00184987, -0.0007911 , ...,  0.00126449,
        0.00274119,  0.00099002]), array([ 0.00611331, -0.00755995,  0.00928848, ...,  0.00349884,
       -0.00017589,  0.0017573 ]), array([-0.00555071,  0.00834524,  0.00230107, ...,  0.00016939,
       -0.00452189, -0.00628126]), array([-0.00285796,  0.00130964, -0.00335708, ..., -0.00152296,
       -0.00401064,  0.00295646]), array([ 0.00196754,  0.00319812, -0.0018926 , ...,  0.00029392,
        0.00116695, -0.00766146]), array([ 0.        , -0.00025405,  0.00463911, ...,  0.00510784,
        0.00122581, -0.00226629]), array([-0.0133608 , -0.0105213 , -0.00107274, ..., -0.00048446,
        0.00952303, 

In [ ]:
scenarios_totaux_TauxdeChange = sum(scenarios_bootstrap_TauxdeChange)
print(scenarios_totaux_TauxdeChange)

SP500 scenarios

In [118]:
scenarios_bootstrap_SP500 = []
for i in range(nb_scenarios):
    new_scenario = np.random.choice(dftaux['Log_Return_SP500'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_SP500.append(new_scenario)
print(scenarios_bootstrap_SP500)

[array([-0.01425506,  0.0005297 , -0.00461625, ...,  0.00365733,
       -0.00396083, -0.00371359]), array([-0.00782974, -0.01133611, -0.00390278, ..., -0.00855377,
       -0.00795406,  0.00069645]), array([ 0.00399933,  0.0004157 , -0.00414346, ...,  0.00981307,
        0.01311384,  0.00526408]), array([-0.00258189, -0.00766012,  0.00396582, ...,  0.00712236,
       -0.00171924,  0.00221952]), array([-0.00201372,  0.00628407,  0.00566886, ...,  0.00032404,
       -0.00180753,  0.00101747]), array([0.00472342, 0.00217257, 0.01590128, ..., 0.00390675, 0.01440407,
       0.00312966]), array([-0.00352301, -0.0022158 ,  0.00029832, ...,  0.00132882,
       -0.00488785,  0.01266134]), array([-0.00180606, -0.00203966, -0.00700536, ..., -0.02108697,
       -0.00565959, -0.00326602]), array([-0.00150229,  0.00292807, -0.01804285, ...,  0.00751069,
       -0.00649556,  0.00467719]), array([-0.00089805, -0.0057976 , -0.0001719 , ..., -0.00031436,
        0.00463352,  0.00909425]), array([-1.62962

In [119]:
scenarios_totaux_SP500 = sum(scenarios_bootstrap_SP500)
print(scenarios_totaux_SP500)

[0.43021012 0.4591971  0.63685505 ... 0.12288641 0.09686277 0.55846799]


***Swap pricing***

Uploading the XML file

In [99]:
xml_data = """
<instrumentlist xmlns="http://www.statpro.net/xml/structure/1.0"><http://www.statpro.net/xml/structure/1.0%22%3e>
  <!-- plain swap -->
  <instrument>
    <code>TEST0183</code>
    <codetype>test-dp</codetype>
    <description>Sample interest-rate swap</description>
    <currency>EUR</currency>
    <swap>
      <counterpartycode>Foo</counterpartycode>
      <fixedrateleg>
        <side>Pay</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>Following</rollingconvention>
        <frequency>1</frequency>
        <adjusted>false</adjusted>
        <daycount>30/360</daycount>
        <notionals>100.0</notionals>
        <couponrates>0.04</couponrates>
      </fixedrateleg>
      <floatingrateleg>
        <side>Receive</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>ModifiedFollowing</rollingconvention>
        <frequency>2</frequency>
        <adjusted>true</adjusted>
        <daycount>Act/360</daycount>
        <notionals>100.0</notionals>
        <referenceindex>EUR-Euribor-6-m</referenceindex>
        <fixingdays>2</fixingdays>
        <fixingtype>Advance</fixingtype>
        <spreads>0.002</spreads>
      </floatingrateleg>
    </swap>
  </instrument>
</instrumentlist>
"""

SWAP constants definition

In [103]:
notional = 100
fixed_rate = 0.04
frequency = 2
maturity = 10
startdate = datetime.strptime("18/03/2008", date_format)
maturitydate = datetime.strptime("17/03/2018", date_format)
market_price = 0  # à remplacer par le prix actuel du swap

In [106]:
# Courbe des taux 
filtered_df = df[['10 Yr']]
rate_curve = list(filtered_df['10 Yr'])
print(rate_curve)

#rate_curve = list(filtered_df.to_records(index=False))
rate_curve = [float(value) for value in rate_curve]

print(rate_curve)

TypeError: list expected at most 1 argument, got 2

Functions definition

In [100]:
# Fonction pour calculer les cash flows de la jambe fixe
def calculate_fixed_leg_coupon(notional, fixed_rate, frequency, maturity):
    fixed_leg_cash_flows = [(fixed_rate / frequency) * notional] * (maturity * frequency)
    return fixed_leg_cash_flows


# Fonction pour extraire l'année d'une date
def extract_year(date):
    return date.year


# Fonction pour calculer les taux forward
def calculate_forward_rates(rate_curve, startdate):
    forward_rates = [((((1 + rate_curve[i]) ** (extract_year(i) - extract_year(startdate))) / ((1 + rate_curve[i - 1]) ** (extract_year(i - 1) - extract_year(startdate)))) ** (extract_year(i) - extract_year(i - 1))) - 1 for i in range(1, len(rate_curve))]
    return forward_rates


# Fonction pour calculer les cash flows de la jambe variable 
def calculate_variable_leg_coupon(forward_rates, notional, frequency, maturity):
    variable_leg_cash_flows = [(rate / frequency) * notional for rate in forward_rates]
    return variable_leg_cash_flows


# Fonction pour calculer le prix du swap
def calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, spread):
    fixed_leg_cash_flows = calculate_fixed_leg_coupon(notional, fixed_rate, frequency, maturity)
    forward_rates = calculate_forward_rates(rate_curve, startdate)
    variable_leg_cash_flows = calculate_variable_leg_coupon(forward_rates, notional, frequency, maturity)

   # Actualisation des cash flows avec le taux d'intérêt et le spread
    discounted_fixed_leg = np.sum([cf / (1 + rate_curve[t] + spread / frequency) ** (t / frequency) for t, cf in enumerate(fixed_leg_cash_flows, start=1)])
    discounted_variable_leg = np.sum([cf / (1 + rate_curve[t]) ** (t / frequency) for t, cf in enumerate(variable_leg_cash_flows, start=1)])

    swap_price = discounted_fixed_leg - discounted_variable_leg
    return swap_price


# Fonction objectif pour la calibration du spread
def objective_function(spread, rate_curve, notional, fixed_rate, frequency, maturity, market_price):
    calculated_price = calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, spread)
    return (calculated_price - market_price) ** 2

In [51]:
# Initialisation du spread
initial_spread = 0.002

# Calibration du spread
calibrated_spread = optimize.minimize(objective_function, initial_spread, args=(rate_curve, notional, fixed_rate, frequency, maturity, market_price)).x[0]

# Calcul du prix du swap avec le spread calibré
final_price = calculate_swap_price(rate_curve, notional, fixed_rate, frequency, maturity, calibrated_spread)

print("Spread calibré:", calibrated_spread)
print("Prix du swap avec spread calibré:", final_price)

AttributeError: 'int' object has no attribute 'year'